In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [53]:
df_16 = pd.read_csv("Trump_2016.csv")
df_18 = pd.read_csv("Trump_2018.csv")

In [54]:
#df_16.columns
#df_16.head(10)
display(df_16.describe(include='all'))
display(df_16.dtypes)
temp = df_16.isnull().sum()
display(temp[temp > 0])

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
count,3649,3649,3648,3648.000000,3648.000000,3648,3.648000e+03
unique,9,3641,3603,NaN,NaN,2,NaN
top,Twitter for iPhone,MAKE AMERICA GREAT AGAIN!,01/10/2016 21:06,NaN,NaN,False,NaN
freq,1950,9,3,NaN,NaN,3461,NaN
mean,NaN,NaN,NaN,8234.224507,22661.954770,NaN,7.441187e+17
std,NaN,NaN,NaN,10423.880893,27375.048677,NaN,3.712391e+16
min,NaN,NaN,NaN,0.000000,0.000000,NaN,6.830000e+17
25%,NaN,NaN,NaN,3501.250000,9328.500000,NaN,7.090000e+17
50%,NaN,NaN,NaN,5956.500000,16499.500000,NaN,7.460000e+17
75%,NaN,NaN,NaN,9991.750000,27048.500000,NaN,7.790000e+17


source             object
text               object
created_at         object
retweet_count     float64
favorite_count    float64
is_retweet         object
id_str            float64
dtype: object

created_at        1
retweet_count     1
favorite_count    1
is_retweet        1
id_str            1
dtype: int64

In [55]:
#Imputing the missing value
#display(df_16[df_16['created_at'].isnull()==True]['text'])

temp = df_16.loc[df_16['created_at'].isnull(), 'text'].values[0]
temp = temp.split(',')

df_16.loc[df_16['created_at'].isnull(), 'text'] = temp[0] 
df_16.loc[df_16['created_at'].isnull(), 'retweet_count'] = float(temp[2])  
df_16.loc[df_16['created_at'].isnull(), 'favorite_count'] = float(temp[3]) 
df_16.loc[df_16['created_at'].isnull(), 'is_retweet'] = temp[4] 
df_16.loc[df_16['created_at'].isnull(), 'id_str'] = float(temp[5])
df_16.loc[df_16['created_at'].isnull(), 'created_at'] = temp[1]
df_16[df_16['created_at'] == temp[1]]


,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
3023,Twitter Ads,This is a crossroads in the history of our civ...,10-13-2016 22:50:04,16977.0,33428.0,false,7.867006e+17


In [65]:
#check nulls again
temp = df_16.isnull().sum()
display(temp[temp > 0])

Series([], dtype: int64)

In [111]:
#class label 1 Android, 0 iPhone
df_16['label'] = np.where(df_16['source'] == 'Twitter for Android', 1, 0)
df_16_train = df_16[df_16['source'].isin(['Twitter for Android', 'Twitter for iPhone'])]


In [112]:
# Text preprocessing 
df_16_train['text'] = df_16_train['text'].apply(lambda x:str(x).lower())
import re
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    #text = re.sub(r'\@\w+|\#', '', text)  # Remove @mentions and hashtags this is reducing accuracy so not removing
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    return text

df_16_train['text'] = df_16_train['text'].apply(clean_text)


C:\Users\DELL\AppData\Local\Temp\ipykernel_17864\2897490961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_16_train['text'] = df_16_train['text'].apply(lambda x:str(x).lower())
C:\Users\DELL\AppData\Local\Temp\ipykernel_17864\2897490961.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_16_train['text'] = df_16_train['text'].apply(clean_text)


In [127]:
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(df_16_train['text'], df_16_train['label'], test_size=0.2)

#TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [128]:
# List of classifiers to test
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
}


results = {}

for name, clf in classifiers.items():
    clf.fit(X_train_tfidf, y_train)
    y_pred = clf.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = {
        "Accuracy": accuracy,
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-Score": report["weighted avg"]["f1-score"]
    }

results_df = pd.DataFrame(results).T
display(results_df.sort_values(by="Accuracy", ascending=False))

,Accuracy,Precision,Recall,F1-Score
SVM,0.824806,0.824443,0.824806,0.824381
Random Forest,0.823256,0.822916,0.823256,0.822691
Logistic Regression,0.813953,0.814359,0.813953,0.812576
Decision Tree,0.775194,0.774931,0.775194,0.773453
Gradient Boosting,0.770543,0.780953,0.770543,0.763363
K-Nearest Neighbors,0.755039,0.755254,0.755039,0.752137


In [129]:
#Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Make predictions and evaluate the classifier
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       362
           1       0.82      0.74      0.78       283

    accuracy                           0.81       645
   macro avg       0.81      0.81      0.81       645
weighted avg       0.81      0.81      0.81       645



In [130]:
# SVC --This give better accuracy
model = SVC()
model.fit(X_train_tfidf, y_train)

# Make predictions and evaluate the classifier
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.85       362
           1       0.81      0.78      0.80       283

    accuracy                           0.82       645
   macro avg       0.82      0.82      0.82       645
weighted avg       0.82      0.82      0.82       645



In [ ]:
#Hyper Parameter tuning
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline

# # Define a pipeline with TF-IDF Vectorizer and Logistic Regression
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(stop_words='english', max_features=3000)),
#     ('logreg', LogisticRegression(max_iter=1000))
# ])

# # Define the hyperparameters grid to search
# param_grid = {
#     'logreg__penalty': ['l1', 'l2', 'elasticnet', 'none'],
#     'logreg__solver': ['liblinear', 'saga'],  
#     'tfidf__max_features': [1000, 3000, 5000] 
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# # Fit the model
# grid_search.fit(X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Hyperparameters:", best_params)
# print("Best Cross-Validation Accuracy:", best_score)

# # Evaluate on the test set
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Test Set Accuracy:", accuracy)


In [120]:
df_16_excluded = df_16[~df_16['source'].isin(['Twitter for Android', 'Twitter for iPhone'])]
df_16_excluded['text'] = df_16_excluded['text'].str.lower()

#Vectorize
X_exclude_tfidf = tfidf.transform(df_16_excluded['text'])

C:\Users\DELL\AppData\Local\Temp\ipykernel_17864\1398221640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_16_excluded['text'] = df_16_excluded['text'].str.lower()


In [121]:
# Prediction for non iphone and android tweets
exclude_pred = model.predict(X_exclude_tfidf)
from collections import Counter
c = Counter(exclude_pred)
print(c) 
#class label 1  (Trump), 0  (not trump)

Counter({0: 302, 1: 122})


2018 dataset

In [122]:
# Apply the classifier to the 2018 data
df_18['text'] = df_18['text'].str.lower()
df_18['text'] = df_18['text'].apply(clean_text)
X_2018_tfidf = tfidf.transform(df_18['text'])
pred_2018 = model.predict(X_2018_tfidf)

In [126]:
c1 = Counter(pred_2018)

print("Prediction 2016:", c)
print("Prediction 2018:", c1)
#class label 1  (Trump), 0  (not trump)

Prediction 2016: Counter({0: 302, 1: 122})
Prediction 2018: Counter({0: 1872, 1: 1684})


In [131]:
1872+1684

3556

#Data Insights
Who writes the tweets that come from the other devices during 2016. How confident can we be in these results?
As our SVM classfier give decent accuracy on both train and test set we can be confident on the prediction but we need moree datapoints to understand the patterns in which trump writes his tweets, with that our model can be improved

● Run the classifier on tweets from Jan-Dec 2018. Do you think this classifier is still
valid, or has something changed [e.g., perhaps Trump is no longer posting tweets]?
How reliable is it for this year?
our classfier predicts that out of 3556 Trump tweeted was fairly active and twetted around 1684 tweets., this may or may not be the case as external factors like sanction are not taking into consideration while training the model hence i think we need to train the model on newer data points + more factors like identify the way trump tweets etc to build a more robust model  